<a href="https://colab.research.google.com/github/eric-huang5497/Math-152/blob/main/Exploration_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def collatz(n):
  for i in range(100):
    print(n)
    n = (n/2 if n%2==0 else 3*n + 1)

collatz(5)

5
16
8.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
2.0
1.0
4.0
